<a href="https://colab.research.google.com/github/alekhyam94/DDL-Project/blob/master/code/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Download Training data
Fetch frame level Youtube 8M dataset

In [2]:
!mkdir -p /content/2/frame/train
os.chdir('/content/2/frame/train')
!curl data.yt8m.org/download.py | shard=1,50 partition=2/frame/train mirror=us python

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0   310k      0 --:--:-- --:--:-- --:--:--  310k
Resuming Download ...
Files remaining 67
Downloading: train0111.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainbX.tfrecord 100.0%Succesfully downloaded train0111.tfrecord 275729962 bytes.
Successfully downloaded train0111.tfrecord


Downloading: train1932.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainFk.tfrecord 100.0%Succesfully downloaded train1932.tfrecord 285925929 bytes.
Successfully downloaded train1932.tfrecord


Downloading: train1138.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainsw.tfrecord 100.0%Succesfully downloaded train1138.tfrecord 276719100 bytes.
Successfully downloaded train1138.tfrecord


Downloading: train2896.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/t

Fetch Frame level validation and training Youtube 8M dataset

In [3]:
!mkdir -p /content/3/frame/test
!mkdir -p /content/3/frame/validate

# Fetch test data
os.chdir('/content/3/frame/test')
!curl data.yt8m.org/download.py | shard=1,50 partition=3/frame/test mirror=us python

# Fetch validation data
os.chdir('/content/3/frame/validate')
!curl data.yt8m.org/download.py | shard=1,50 partition=3/frame/validate mirror=us python

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0  68461      0 --:--:-- --:--:-- --:--:-- 68461
Resuming Download ...
Files remaining 81
Downloading: test3104.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/testYe.tfrecord 100.1%Succesfully downloaded test3104.tfrecord 3543552 bytes.
Successfully downloaded test3104.tfrecord


Downloading: test3417.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/test3h.tfrecord 100.2%Succesfully downloaded test3417.tfrecord 3458142 bytes.
Successfully downloaded test3417.tfrecord


Downloading: test3639.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/test6R.tfrecord 100.2%Succesfully downloaded test3639.tfrecord 703368 bytes.
Successfully downloaded test3639.tfrecord


Downloading: test2072.tfrecord
>> Downloading http://us.data.yt8m.org/3/frame/test/testHA.tfrecord 100.3%Suc

# Training
Train LSTM Model on Frame Level features

In [4]:
os.chdir('/content/drive/My Drive/youtube-8m-code/code/youtube-8m/')
!python train.py --frame_features --model=LstmModel --feature_names='rgb,audio' --feature_sizes='1024,128' --train_data_pattern=/content/2/frame/train/train*.tfrecord --train_dir=/content/models/frame/lstm_50 --start_new_model --num_epochs=5 --batch_size=128 --base_learning_rate=0.0002 --iterations=150

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




W1117 06:12:09.972982 139876857042816 module_wrapper.py:139] From train.py:689: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1117 06:12:09.973162 139876857042816 module_wrapper.py:139] From train.py:689: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1117 06:12:09.973278 139876857042816 module_wrapper.py:139] From train.py:690: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.15.0.
I1117 06:12:09.973347 139876857042816 train.py:6

# Evaluation

Get evaluation metrics value on Validation dataset

In [5]:
os.chdir('/content/drive/My Drive/youtube-8m-code/code/youtube-8m/')
!python eval.py -eval_data_pattern=/content/3/frame/validate/validate*.tfrecord --train_dir /content/models/frame/lstm_50 --segment_labels --run_once True

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I1117 06:54:04.212535 140683551942528 eval.py:352] tensorflow version: 1.15.0
I1117 06:54:04.218971 140683551942528 eval.py:87] Using batch size of 1024 for evaluation.
I1117 06:54:04.224055 140683551942528 eval.py:92] number of evaluation files: 76
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W1117 06:54:04.224260 140683551942528 deprecation.py:323] From eval.py:95: string_input_producer (from tensorflow.python.t

# Visualization

In [7]:
os.chdir('/content/models/frame/')
%load_ext tensorboard
%tensorboard --logdir lstm_50

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
